In [3]:
import tantivy
import json

In [5]:
schema_builder = tantivy.SchemaBuilder()
schema_builder.add_text_field("body", stored=True)
schema_builder.add_text_field("doc_id", stored=True)
schema = schema_builder.build()
index = tantivy.Index(schema, path="data/fiqa/bm25.tantivy/")


In [6]:
searcher = index.searcher()

In [7]:
import re
def sanitize_query_for_tantivy(query):
    # remove special characters: ()[]{}^"~*?:
    query = re.sub(r'[\(\)\[\]\{\}\^\"\~\*\?\:\`]', ' ', query)
    return query


def search_bm25(query, limit):
    query = index.parse_query(sanitize_query_for_tantivy(query), ['body'])
    hits = searcher.search(query, limit).hits
    docs = [
        {
            "body": searcher.doc(doc_address)["body"],
            "doc_id": searcher.doc(doc_address)["doc_id"],
            "score": score
        }
        for (score, doc_address) in hits
    ]
    return docs

In [8]:
query = "What are the consequences of IRS “reclassification” on both employer and employee?"

for hit in search_bm25(query, 10):
    print(hit)

{'body': ["In the USA, you probably owe Self Employment Tax. The cutoff for tax on this is 400$. You will need to file a tax return and cover the medicaid expenses as if you were both the employer and employee.  In addition, if he earns income from self-employment, he may owe Self-Employment Tax, which means paying both the employee’s and employer's share of Social Security and Medicaid taxes. The trigger for Self Employment Tax has been $400 since 1990, but the IRS may change that in the future. Also see the IRS website.  So yes, you need to file your taxes. How much you will pay is determined by exactly how much your income is.  If you don't file, you probably won't be audited, however you are breaking the law and should be aware of the consequences."], 'doc_id': ['89611'], 'score': 27.17253875732422}
{'body': ['"You are confusing entirely unrelated things. First the ""profit distribution"" issue with Bob\'s S-Corp which is in fact tax evasion and will probably trigger a very nasty a